#  Imports and Funcs

##  Imports

In [3]:
import os

In [4]:
import re

In [5]:
import pandas as pd
import numpy as np
from pandas.api.types import is_string_dtype, is_numeric_dtype

In [6]:
pd.set_option('display.max_colwidth', None)

In [7]:
import pickle

In [8]:
import time

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [10]:
from sklearn.neighbors import NearestNeighbors

In [11]:
from ftfy import fix_text

In [130]:
import networkx as nx

## Functions

In [12]:
def pickleOut(obj, fname):
    fout =  open(fname, 'wb')
    pickle.dump(obj, fout)
    fout.close()
    print(f'object was successfully exported to {fname}')

In [13]:
def pickleIn(fname):
    fIn =  open(fname, 'rb')
    obj = pickle.load(fIn)
    fIn.close()
    print(f'object {fname} was successfully imported  ')
    return obj

# Todos 

In [14]:
def fixStrTextNorm(dstr):
    dstr = fix_text(dstr)
    dstr = dstr.lower()
    dstr = dstr.replace('|', '')
    dstr = dstr.strip()
    return dstr 

def dfObjColConverter(df, ltcols):
    for col in ltcols:
        if df[col].dtype == 'object':
            df[col] = df[col].fillna('') 
            df[col]  =  df[col].astype('str').apply(fixStrTextNorm)
    return df

def compNameGenerator(df, ltcols, sep = '| ' ):    
    return  df[ltcols].apply(lambda x: sep.join(x.dropna().astype(str).values), axis=1) 

def ngrams(string, n=3):
    string = re.sub(r'[,-./|]',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

In [126]:
def dfMatchesTodDupID(df):
    g = nx.from_pandas_edgelist(df, 'DatabaseData', 'QueryData')
    ltconx = [{'compname': list(it)} for it in nx.connected_components(g)]
    dfgroup = pd.DataFrame(ltconx)
    dfgroup['groupID'] = dfgroup.index
    dfduptracker = dfgroup.explode('compname')
    return dfduptracker 

# Code

In [92]:
ltColumnsToMatch  = ['first_name', 'mid_name', 'last_name']

In [123]:
def fuzzMatch(ltColumnsToMatch):
    df = pd.read_csv('upload.csv')
    df =  dfObjColConverter(df, ltColumnsToMatch)
    df['compname'] = compNameGenerator(df, ltColumnsToMatch)
    dfdb =  df.compname.drop_duplicates()
    vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
    db_tf_idf_matrix = vectorizer.fit_transform(dfdb)
    knn = NearestNeighbors(n_neighbors=10 , metric = 'cosine' )
    knn.fit(db_tf_idf_matrix)
    D, I = knn.kneighbors(db_tf_idf_matrix, 10)  

    matches = []
    for r,indVals in enumerate(I):
        for c, dbloc in enumerate(indVals):
            temp = [D[r][c], dfdb.iloc[dbloc], dfdb.iloc[r]]
            matches.append(temp)

    dfmatches = pd.DataFrame(matches, columns = ['Kdistance','DatabaseData','QueryData'])\
                .sort_values('Kdistance', ascending = True)
    return dfmatches

In [124]:
dfFuzzMatch = fuzzMatch(ltColumnsToMatch)

In [120]:
thold = 0.25

In [125]:
dfFuzzMatch[(dfFuzzMatch['Kdistance'] < thold)  &  (dfFuzzMatch['Kdistance'] > 0.000000001)]

,Kdistance,DatabaseData,QueryData
31111,0.030607,maria carmen gladys| amante| pitogo,ma. carmen gladys| amante| pitogo
31101,0.030607,ma. carmen gladys| amante| pitogo,maria carmen gladys| amante| pitogo
42301,0.039898,ronaliza| abulencia| sablayan,monaliza| abulencia| sablayan
42311,0.039898,monaliza| abulencia| sablayan,ronaliza| abulencia| sablayan
561,0.053098,aida| palami| sandangan,h.aida| palami| sandangan
...,...,...,...
10231,0.249566,jolito| petalco| melloria,julieto| petalco| melloria
12061,0.249627,evangeline| dela cruz| cruz,evangeline| dela cruz| mendoza
12033,0.249627,evangeline| dela cruz| mendoza,evangeline| dela cruz| cruz
6481,0.249656,renato| dela cruz| dela cruz,renato| de villa| dela cruz


#### Next nat tong sa baba 

In [131]:
0dftracker =  dfMatchesTodDupID(dfFuzzMatch[(dfFuzzMatch['Kdistance'] < thold)])

In [132]:
dftracker

,compname,groupID
0,jocelyn| pascua| dela cruz,0
0,joselyn| pascua| dela cruz,0
0,jennelyn| pascua| dela cruz,0
1,taiba| dadih| asjah,1
2,tambai| kalim| sultan,2
...,...,...
5056,roland| mesiano| matilla,5056
5057,marites| de guzman| salvador,5057
5058,angelita| lapitag| jabalde,5058
5059,imelda| dela cruz| agustin,5059


In [133]:
df = pd.read_csv('upload.csv')
df =  dfObjColConverter(df, ltColumnsToMatch)
df['compname'] = compNameGenerator(df, ltColumnsToMatch)

In [137]:
dfDups = dftracker.merge(df, on = 'compname', how = 'inner')

In [138]:
dfDups

,compname,groupID,No,hh_id,payroll_row_id,payroll_transaction_log_id,payroll_batch_id,hh_id.1,psgc_region,psgc_province,...,region_branch,date_claimed,claim_status,recon_remarks,rice_amount,date_inserted,time_stamp,adjustment_amount,gross_amount,net_amount
0,jocelyn| pascua| dela cruz,0,20215,015502003-10333-00058,504529338.0,1.613471e+14,PP-37E01120120210201,015502003-10333-00058,3.0,10.0,...,NaN,NaN,0.0,1.0,600.0,2021-02-16,2021-03-15 15:02:40,0.0,0.0,0.0
1,jocelyn| pascua| dela cruz,0,20215,015502003-10333-00058,512811502.0,1.618843e+14,PP-37E01020321210400,015502003-10333-00058,3.0,10.0,...,NaN,NaN,0.0,1.0,600.0,2021-04-19,2021-05-31 13:38:09,0.0,0.0,0.0
2,jocelyn| pascua| dela cruz,0,20215,015502003-10333-00058,512815934.0,1.618843e+14,PP-37E01020321210400,015502003-10333-00058,3.0,10.0,...,NaN,NaN,0.0,1.0,600.0,2021-04-19,2021-05-31 13:38:09,0.0,0.0,0.0
3,jocelyn| pascua| dela cruz,0,20215,015502003-10333-00058,520813630.0,1.623769e+14,PP-37E01040521210600,015502003-10333-00058,3.0,10.0,...,NaN,NaN,0.0,1.0,600.0,2021-06-15,2021-09-12 14:10:44,0.0,0.0,0.0
4,jocelyn| pascua| dela cruz,0,20215,015502003-10333-00058,520818079.0,1.623769e+14,PP-37E01040521210600,015502003-10333-00058,3.0,10.0,...,NaN,NaN,0.0,1.0,600.0,2021-06-15,2021-09-12 14:10:44,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355685,jocelyn| dizon| flores,5060,19955,064520012-4500-00058,326430932.0,1.508403e+14,PP-95A027080917171000,064520012-4500-00058,9.0,42.0,...,586.0,2017-12-01,3.0,6.0,600.0,2017-10-19,2020-05-29 14:13:23,0.0,0.0,0.0
355686,jocelyn| dizon| flores,5060,19955,064520012-4500-00058,326446989.0,1.508403e+14,PP-95A027080917171000,064520012-4500-00058,9.0,42.0,...,586.0,2017-12-01,3.0,6.0,600.0,2017-10-19,2020-05-29 14:13:23,0.0,0.0,0.0
355687,jocelyn| dizon| flores,5060,19955,064520012-4500-00058,334490247.0,1.513696e+14,PP-95A027101117171200,064520012-4500-00058,9.0,42.0,...,NaN,NaN,0.0,1.0,600.0,2017-12-19,2018-01-11 14:22:15,0.0,0.0,0.0
355688,jocelyn| dizon| flores,5060,19955,064520012-4500-00058,334506331.0,1.513696e+14,PP-95A027101117171200,064520012-4500-00058,9.0,42.0,...,NaN,NaN,0.0,1.0,600.0,2017-12-19,2018-01-11 14:22:15,0.0,0.0,0.0


# Test 

# 

# 